# Inconsistency Table (MORPH)

In [1]:
%load_ext watermark

In [2]:
watermark -a 'Sebastian Raschka' -p torch,numpy,pandas

Sebastian Raschka 

torch 1.5.1
numpy 1.18.1
pandas 1.0.1


In [3]:
import torch
import numpy as np
import pandas as pd
import os

In [4]:
def count_inconsistency(ary):
    cnt = 0
    last = ary[0]
    for i in ary[1:]:
        if i > last + 1e-05:
            cnt += 1
        last = i
    return cnt


def read_labels_from_log(path):
    with open(path, 'r') as f:
        cont = f.read()
    return np.array(cont.split(',')).astype(int)

In [5]:
basepath = '../experiment-logs/morph2'
morph_test_path = '../datasets/morph2_test.csv'

In [6]:
morph_test_labels = pd.read_csv(morph_test_path, usecols=['age']).values.reshape(-1)
morph_test_labels[:15]

array([ 6, 16, 38, 29, 30, 29, 19, 28, 11,  5, 21,  7, 15,  1, 22])

In [7]:
# Coral

for i in range(3):
    
    morph_ordinal_path = os.path.join(basepath, f'morph-coral__seed{i}')
    
    test_proba_path_seed00 = os.path.join(morph_ordinal_path, 'test_allprobas.tensor')
    t = torch.load(test_proba_path_seed00)

    inconsistency_counts = [count_inconsistency(t[i]) for i in range(t.size()[0])]
    avg = np.mean(inconsistency_counts)
    print(f'Average Inconsistency Seed {i}: {avg:.2f}')
    

Average Inconsistency Seed 0: 0.00
Average Inconsistency Seed 1: 0.00
Average Inconsistency Seed 2: 0.00


In [8]:
# Ordinal

for i in range(3):
    
    morph_ordinal_path = os.path.join(basepath, f'morph-ordinal__seed{i}')
    
    test_proba_path_seed00 = os.path.join(morph_ordinal_path, 'test_allprobas.tensor')
    t = torch.load(test_proba_path_seed00)

    inconsistency_counts = [count_inconsistency(t[i]) for i in range(t.size()[0])]
    avg = np.mean(inconsistency_counts)
    print(f'Average Inconsistency Seed {i}: {avg:.2f}')
    

Average Inconsistency Seed 0: 2.28
Average Inconsistency Seed 1: 2.08
Average Inconsistency Seed 2: 0.86


In [9]:
for i in range(0, 3):

    when_correct_cnt = []
    when_incorrect_cnt = []
    
    morph_ordinal_path = os.path.join(basepath, f'morph-ordinal__seed{i}')

    test_proba_path_seed = os.path.join(morph_ordinal_path, 'test_allprobas.tensor')
    t = torch.load(test_proba_path_seed)
    
    test_proba_path = os.path.join(morph_ordinal_path, 'test_predictions.log')
    lab = read_labels_from_log(test_proba_path)    
    
    for j in range(morph_test_labels.shape[0]):
        if morph_test_labels[j] != lab[j]:
            when_incorrect_cnt.append(count_inconsistency(t[j]))
        else:
            when_correct_cnt.append(count_inconsistency(t[j]))
            
    avg = np.mean(when_correct_cnt)
    print(f'Seed 0{i} when correct: {avg:.2f}')
    
    avg = np.mean(when_incorrect_cnt)
    print(f'Seed 0{i} when incorrect: {avg:.2f}')

Seed 00 when correct: 1.80
Seed 00 when incorrect: 2.37
Seed 01 when correct: 1.70
Seed 01 when incorrect: 2.15
Seed 02 when correct: 0.65
Seed 02 when incorrect: 0.89
